In [1]:
%run '/mnt/d/Users/Admin/Projects/Machine_Learning/Yu_Xian/affine.py'
from osgeo import gdal

In [2]:
gcps = []
gcps_lst = []
for _ in range(4):
    px = np.random.uniform(0, 799)
    py = np.random.uniform(0, 599)

    X = np.random.uniform(-180, 180)
    Y = np.random.uniform(-90, 90)
    
    gcps_lst.append((X, Y, px, py))
    
    gcp = gdal.GCP(X, Y, 0, px, py)
    gcps.append(gcp)

In [3]:
for i, gcp in enumerate(gcps):
    print(f"GCP {i}:")
    print(f"  pixel,line = ({gcp.GCPPixel}, {gcp.GCPLine})")
    print(f"  X,Y,Z      = ({gcp.GCPX}, {gcp.GCPY}, {gcp.GCPZ})")

GCP 0:
  pixel,line = (298.5052442578666, 369.4451749380869)
  X,Y,Z      = (136.3464903296407, -8.172396352092392, 0.0)
GCP 1:
  pixel,line = (772.3790094211295, 475.7981762871681)
  X,Y,Z      = (14.834522162584818, 87.09167899023805, 0.0)
GCP 2:
  pixel,line = (712.0402511184003, 386.5102943412619)
  X,Y,Z      = (-147.2296385775714, -12.600456542968317, 0.0)
GCP 3:
  pixel,line = (455.63004326667823, 455.4126652310397)
  X,Y,Z      = (150.44307202542416, -26.227470528228274, 0.0)


In [4]:
gt = gdal.GCPsToGeoTransform(gcps)

In [5]:
gt

(-369.67116042142743,
 -0.6855171076204924,
 1.8774951268247895,
 -205.62378925681872,
 0.104409061823628,
 0.3727321535317781)

In [6]:
res_lst = []
for (X, Y, px, py) in gcps_lst:
    pred_X, pred_Y = transform( (px, py), gt[0:3], gt[3:] )
    res_x = (pred_X - X)**2
    res_y = (pred_Y - Y)**2
    res = np.sqrt(res_x + res_y)
    print( (X, Y, px, py) )
    print(pred_X, pred_Y)
    res_lst.append(res)

(136.3464903296407, -8.172396352092392, 298.5052442578666, 369.4451749380869)
119.32990350056116 -36.75304108782433
(14.834522162584818, 87.09167899023805, 772.3790094211295, 475.7981762871681)
-5.841427615300972 52.364857383113275
(-147.2296385775714, -12.600456542968317, 712.0402511184003, 386.5102943412619)
-132.11573978414822 12.784479715133898
(150.44307202542416, -26.227470528228274, 455.63004326667823, 455.4126652310397)
173.02170983896576 11.695059556526303


In [7]:
res_lst

[np.float64(33.262854369727876),
 np.float64(40.4159255510835),
 np.float64(29.543610570912808),
 np.float64(44.13516934989835)]

In [ ]:
gcps_lst_cp, E12, N12, E21, N21 = py_GCPTransformer(gcps_lst, False, -1, 3)

In [11]:
py_res_lst = []
for (X, Y, px, py) in gcps_lst_cp:
    pred_X, pred_Y = transform( (px, py), E12, N12 )
    res_x = (pred_X - X)**2
    res_y = (pred_Y - Y)**2
    res = np.sqrt(res_x + res_y)
    print( (X, Y, px, py) )
    print(pred_X, pred_Y)
    py_res_lst.append(res)
py_res_lst

(136.3464903296407, -8.172396352092392, 298.5052442578666, 369.4451749380869)
119.32990350056127 -36.75304108782436
(14.834522162584818, 87.09167899023805, 772.3790094211295, 475.7981762871681)
-5.841427615301086 52.364857383113176
(-147.2296385775714, -12.600456542968317, 712.0402511184003, 386.5102943412619)
-132.11573978414822 12.784479715133898
(150.44307202542416, -26.227470528228274, 455.63004326667823, 455.4126652310397)
173.02170983896576 11.695059556526218


[np.float64(33.26285436972784),
 np.float64(40.41592555108364),
 np.float64(29.543610570912808),
 np.float64(44.13516934989828)]

# Old Code

In [ ]:
# This is what CRS_compute_georef_equations does

def solve_lss_from_gcps(gcps_lst):

    # solves pixel to georef first
    # matrix A12 is from pixel to georef
    A12 = np.array([[1, px, py] for (X, Y, px, py) in gcps_lst])
    bX12 = np.array([X for (X, Y, px, py) in gcps_lst])
    bY12 = np.array([Y for (X, Y, px, py) in gcps_lst])

    E12, *_ = np.linalg.lstsq(A12, bX12, rcond=None)
    N12, *_ = np.linalg.lstsq(A12, bY12, rcond=None)

    # solves georef to pixel
    A21 = np.array([[1, X, Y] for (X, Y, px, py) in gcps_lst])
    bX21 = np.array([px for (X, Y, px, py) in gcps_lst])
    bY21 = np.array([py for (X, Y, px, py) in gcps_lst])

    E21, *_ = np.linalg.lstsq(A21, bX21, rcond=None)
    N21, *_ = np.linalg.lstsq(A21, bY21, rcond=None)
    
    return E12, N12, E21, N21

# This is what CRS_georef does

def transform(coords, E, N):
    x, y = coords[0], coords[1]

    X = E[0] + E[1] * x + E[2] * y
    Y = N[0] + N[1] * x + N[2] * y
    return X, Y

def py_worst_outlier(gcps_lst, E, N, dfTolerance):

    worst_index = -1
    worst_residual = -1.0

    for i, gcp in enumerate(gcps_lst):
        X, Y, px, py = gcp

        X_pred, Y_pred = transform((px, py), E, N)
        X_res = (X_pred - X)
        Y_res = (Y_pred - Y)
    
        Res = np.sqrt((X_res)**2 + (Y_res)**2)
        if (Res >= dfTolerance) and (Res > worst_residual):
            worst_residual = Res
            worst_index = i

    if worst_index == -1:
        return "No outliers"
    else:
        return worst_index

def py_rm_outliers(gcps_lst, dfTolerance, nGCPmin = 3):

    gcps_lst_cp = list(gcps_lst)
    E12, N12, E21, N21 = solve_lss_from_gcps(gcps_lst_cp)

    while len(gcps_lst_cp) > nGCPmin:
        worst_index = py_worst_outlier(gcps_lst_cp, E12, N12, dfTolerance)
        if worst_index == "No outliers":
            break
        else:
            del gcps_lst_cp[worst_index]
            E12, N12, E21, N21 = solve_lss_from_gcps(gcps_lst_cp)

    return gcps_lst_cp, E12, N12, E21, N21

def py_GCPTransformer(gcps_lst, bRefine, dfTolerance, nGCPmin = 3):

    if (bRefine == True):
        if nGCPmin < 4:
            return "Need 4 or more GCPs for bRefine to be True"
        else:
            gcps_lst_cp, E12, N12, E21, N21 = py_rm_outliers(gcps_lst, dfTolerance, nGCPmin)
            return gcps_lst_cp, E12, N12, E21, N21
    else:
        E12, N12, E21, N21 = solve_lss_from_gcps(gcps_lst)
        return gcps_lst, E12, N12, E21, N21